# Анализ приложения для чтения книг
Во время коронавируса сильно изменился привычный уклад жизни, люди стали гораздо больше времени проводить дома и больше читать. Наша компания решила не упускать возможость развития и преобрела сервис для чтения книг по подписке. 
# Цель
Необходимо проанализировать базу данных, чтобы сформировать ценностные предложения для нового продукта компании.
# Описание данных
**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Знакомство с данными

In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
pd.io.sql.read_sql("SELECT * FROM books LIMIT 3", con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [4]:
pd.io.sql.read_sql("SELECT * FROM authors LIMIT 3", con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


In [5]:
pd.io.sql.read_sql("SELECT * FROM publishers LIMIT 3", con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


In [6]:
pd.io.sql.read_sql("SELECT * FROM ratings LIMIT 3", con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [7]:
pd.io.sql.read_sql("SELECT * FROM reviews LIMIT 3", con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


Данные просмотрены, можно переходить к анализу.

## Анализ данных
### Cколько книг вышло после 1 января 2000 года?

In [8]:
bo = '''
SELECT COUNT(*)    
FROM books 
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(bo, con = engine)

,count
0,819


ОТВЕТ:
    
819 книг

### Для каждой книги посчитем количество обзоров и среднюю оценку

In [9]:
bo_re_ra = '''
WITH
c_r AS (SELECT book_id,
        COUNT(review_id) AS count_review   
        FROM reviews 
        GROUP BY book_id),
        
m_r AS (SELECT book_id,
       AVG(rating) AS mean_rating   
FROM ratings 
GROUP BY book_id)
        
SELECT title AS book,
       c_r.count_review,
       m_r.mean_rating
FROM books 
LEFT JOIN c_r ON books.book_id=c_r.book_id
LEFT JOIN m_r ON books.book_id=m_r.book_id
'''
pd.io.sql.read_sql(bo_re_ra, con = engine)

,book,count_review,mean_rating
0,The Body in the Library (Miss Marple #3),2.0,4.500000
1,Galápagos,2.0,4.500000
2,A Tree Grows in Brooklyn,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,The Prophet,4.0,4.285714
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.000000
996,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.666667
997,Essential Tales and Poems,NaN,4.000000
998,Leonardo's Notebooks,NaN,4.000000


### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [10]:
pub = '''
WITH
p_id AS (SELECT publisher_id,
                COUNT(book_id) AS count_book
         FROM books 
         WHERE num_pages > 50
         GROUP BY publisher_id)
         
SELECT publisher,
       p_id.count_book
FROM publishers
JOIN p_id ON publishers.publisher_id=p_id.publisher_id
ORDER BY count_book DESC;
'''
pd.io.sql.read_sql(pub, con = engine)

,publisher,count_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Ballantine Books (NY),1
330,Plaza y Janés,1
331,HarperCollinsPublishers,1
332,Random House: Modern Library,1


<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid " > <b>Комментарии от ревьюера ✔️ : </b> 
    

Согласен с результатом. Действительно данное издательство является лидером по выпущенным книгам

</div>

ОТВЕТ:
    
Издательство Penguin Books выпустило 42 книги толще 50 страниц

### Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [11]:
au = '''
SELECT a.author,
       AVG(rat.mean_rat) AS mean_rat
FROM
    (SELECT book_id,
            AVG(rating) AS mean_rat
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id)>=50) AS rat
LEFT JOIN books AS b ON rat.book_id=b.book_id
LEFT JOIN authors AS a ON b.author_id=a.author_id
GROUP BY a.author
ORDER BY mean_rat DESC
'''
pd.io.sql.read_sql(au, con = engine)

,author,mean_rat
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


ОТВЕТ:

Автор J.K. Rowling/Mary GrandPré получил самую высоку среднюю оценку книг (4.283844), учитывая только книги с 50 и более оценками

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
us = '''
SELECT AVG(count_review)
FROM
    (SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id)>50) AS u
LEFT JOIN 
    (SELECT COUNT(review_id) AS count_review,
            username
     FROM reviews
     GROUP BY username) AS c ON u.username=c.username
'''
pd.io.sql.read_sql(us, con = engine)

,avg
0,24.333333


ОТВЕТ:

24,3 среднее количество обзоров от пользователей, которые поставили больше 50 оценок

## Заключение
- ***819*** книг вышло после 1 января 2000 года
- Для каждой книги посчитано количество обзоров и средняя оценка
- Издательство `Penguin Books` выпустило ***42*** книги толще 50 страниц
- Автор `J.K. Rowling/Mary GrandPré` получил самую высоку среднюю оценку книг (***4.283844***), учитывая только книги с 50 и более оценками
- ***24,3*** среднее количество обзоров от пользователей, которые поставили больше 50 оценок